In [1]:
%matplotlib inline


FastText Model
==============

Introduces Gensim's fastText model and demonstrates its use on the Lee Corpus.



In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Here, we'll learn to work with fastText library for training word-embedding
models, saving & loading them and performing similarity operations & vector
lookups analogous to Word2Vec.



When to use FastText?
---------------------

The main principle behind `fastText <https://github.com/facebookresearch/fastText>`_ is that the morphological structure of a word carries important information about the meaning of the word, which is not taken into account by traditional word embeddings, which train a unique word embedding for every individual word. This is especially significant for morphologically rich languages (German, Turkish) in which a single word can have a large number of morphological forms, each of which might occur rarely, thus making it hard to train good word embeddings.


fastText attempts to solve this by treating each word as the aggregation of its subwords. For the sake of simplicity and language-independence, subwords are taken to be the character ngrams of the word. The vector for a word is simply taken to be the sum of all vectors of its component char-ngrams.


According to a detailed comparison of Word2Vec and FastText in `this notebook <https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Word2Vec_FastText_Comparison.ipynb>`__, fastText does significantly better on syntactic tasks as compared to the original Word2Vec, especially when the size of the training corpus is small. Word2Vec slightly outperforms FastText on semantic tasks though. The differences grow smaller as the size of training corpus increases.


Training time for fastText is significantly higher than the Gensim version of Word2Vec (\ ``15min 42s`` vs ``6min 42s`` on text8, 17 mil tokens, 5 epochs, and a vector size of 100).


fastText can be used to obtain vectors for out-of-vocabulary (OOV) words, by summing up vectors for its component char-ngrams, provided at least one of the char-ngrams was present in the training data.




Training models
---------------




For the following examples, we'll use the Lee Corpus (which you already have if you've installed gensim) for training our model.






In [24]:
from pprint import pprint as print
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

In [25]:
# Set file names for train and test data
corpus_file = datapath('lee_background.cor')

model = FT_gensim(size=100)

2020-03-10 21:48:52,594 : INFO : resetting layer weights


In [26]:
# build the vocabulary
model.build_vocab(corpus_file=corpus_file)

2020-03-10 21:49:00,819 : INFO : collecting all words and their counts
2020-03-10 21:49:00,820 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-03-10 21:49:00,835 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2020-03-10 21:49:00,835 : INFO : Loading a fresh vocabulary
2020-03-10 21:49:01,003 : INFO : effective_min_count=5 retains 1762 unique words (16% of original 10781, drops 9019)
2020-03-10 21:49:01,004 : INFO : effective_min_count=5 leaves 46084 word corpus (76% of original 59890, drops 13806)
2020-03-10 21:49:01,009 : INFO : deleting the raw counts dictionary of 10781 items
2020-03-10 21:49:01,010 : INFO : sample=0.001 downsamples 45 most-common words
2020-03-10 21:49:01,010 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2020-03-10 21:49:01,026 : INFO : estimated required memory for 1762 words, 16936 buckets and 100 dimensions: 9411496 bytes
2020-03-10 21:49:01,027 : INFO : res

In [27]:
# train the model
model.train(
    corpus_file=corpus_file, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words
)

2020-03-10 21:49:10,372 : INFO : training model with 3 workers on 1762 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-03-10 21:49:10,463 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-03-10 21:49:10,465 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-03-10 21:49:10,467 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-03-10 21:49:10,467 : INFO : EPOCH - 1 : training on 60387 raw words (32958 effective words) took 0.1s, 363784 effective words/s
2020-03-10 21:49:10,566 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-03-10 21:49:10,568 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-03-10 21:49:10,578 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-03-10 21:49:10,579 : INFO : EPOCH - 2 : training on 60387 raw words (32906 effective words) took 0.1s, 307599 effective words/s
2020-03-10 21:49:10,669 : INFO : worker

In [28]:
print(model)

Training hyperparameters
^^^^^^^^^^^^^^^^^^^^^^^^




Hyperparameters for training the model follow the same pattern as Word2Vec. FastText supports the following parameters from the original word2vec:

- model: Training architecture. Allowed values: `cbow`, `skipgram` (Default `cbow`)
- size: Size of embeddings to be learnt (Default 100)
- alpha: Initial learning rate (Default 0.025)
- window: Context window size (Default 5)
- min_count: Ignore words with number of occurrences below this (Default 5)
- loss: Training objective. Allowed values: `ns`, `hs`, `softmax` (Default `ns`)
- sample: Threshold for downsampling higher-frequency words (Default 0.001)
- negative: Number of negative words to sample, for `ns` (Default 5)
- iter: Number of epochs (Default 5)
- sorted_vocab: Sort vocab by descending frequency (Default 1)
- threads: Number of threads to use (Default 12)


In addition, FastText has three additional parameters:

- min_n: min length of char ngrams (Default 3)
- max_n: max length of char ngrams (Default 6)
- bucket: number of buckets used for hashing ngrams (Default 2000000)


Parameters ``min_n`` and ``max_n`` control the lengths of character ngrams that each word is broken down into while training and looking up embeddings. If ``max_n`` is set to 0, or to be lesser than ``min_n``\ , no character ngrams are used, and the model effectively reduces to Word2Vec.



To bound the memory requirements of the model being trained, a hashing function is used that maps ngrams to integers in 1 to K. For hashing these character sequences, the `Fowler-Noll-Vo hashing function <http://www.isthe.com/chongo/tech/comp/fnv>`_ (FNV-1a variant) is employed.




**Note:** As in the case of Word2Vec, you can continue to train your model while using Gensim's native implementation of fastText.




Saving/loading models
---------------------




Models can be saved and loaded via the ``load`` and ``save`` methods.




In [4]:
# saving a model trained via Gensim's fastText implementation
import tempfile
import os
with tempfile.NamedTemporaryFile(prefix='saved_model_gensim-', delete=False) as tmp:
    model.save(tmp.name, separately=[])

loaded_model = FT_gensim.load(tmp.name)
print(loaded_model)

os.unlink(tmp.name)

2020-03-10 21:41:41,207 : INFO : saving FastText object under C:\Users\user\AppData\Local\Temp\saved_model_gensim-7lhybykg, separately []
2020-03-10 21:41:41,208 : INFO : storing np array 'vectors_ngrams' to C:\Users\user\AppData\Local\Temp\saved_model_gensim-7lhybykg.wv.vectors_ngrams.npy
2020-03-10 21:41:43,147 : INFO : not storing attribute vectors_norm
2020-03-10 21:41:43,148 : INFO : not storing attribute vectors_vocab_norm
2020-03-10 21:41:43,149 : INFO : not storing attribute vectors_ngrams_norm
2020-03-10 21:41:43,149 : INFO : not storing attribute buckets_word
2020-03-10 21:41:43,150 : INFO : storing np array 'vectors_ngrams_lockf' to C:\Users\user\AppData\Local\Temp\saved_model_gensim-7lhybykg.trainables.vectors_ngrams_lockf.npy
2020-03-10 21:41:49,713 : INFO : saved C:\Users\user\AppData\Local\Temp\saved_model_gensim-7lhybykg
2020-03-10 21:41:49,714 : INFO : loading FastText object from C:\Users\user\AppData\Local\Temp\saved_model_gensim-7lhybykg
2020-03-10 21:41:49,742 : IN

The ``save_word2vec_method`` causes the vectors for ngrams to be lost. As a result, a model loaded in this way will behave as a regular word2vec model.




Word vector lookup
------------------


**Note:** Operations like word vector lookups and similarity queries can be performed in exactly the same manner for both the implementations of fastText so they have been demonstrated using only the native fastText implementation here.



FastText models support vector lookups for out-of-vocabulary words by summing up character ngrams belonging to the word.




In [5]:
print('night' in model.wv.vocab)

True


In [6]:
print('nights' in model.wv.vocab)

False


In [7]:
print(model['night'])

array([ 9.49415863e-02,  5.90373576e-03, -5.72208643e-01,  4.88739759e-01,
        5.98190427e-01, -3.24555606e-01, -1.84535354e-01, -3.69616263e-02,
        4.30120021e-01,  3.38728935e-01, -6.35988057e-01, -1.13690756e-02,
       -6.64123476e-01,  4.12354976e-01,  2.88462251e-01, -6.14066683e-02,
       -1.87915862e-01,  1.82341829e-01,  2.39225477e-01, -3.75126839e-01,
       -2.33614624e-01,  2.77896821e-01, -3.78508836e-01,  1.73061173e-02,
       -8.30545664e-01,  7.31233835e-01,  1.13920026e-01,  1.73350975e-01,
        4.07674164e-01,  3.57048376e-03, -6.53613985e-01,  2.31604159e-01,
        8.53485018e-02, -4.74979430e-01,  4.63648021e-01,  1.14711665e-01,
       -1.62452549e-01, -7.59977624e-02,  4.28185731e-01,  2.23893389e-01,
       -8.57773237e-03, -7.43460655e-02,  3.91313285e-01, -6.72177300e-02,
        1.26174882e-01,  1.87195867e-01, -1.37181431e-01,  2.17835397e-01,
       -2.08233222e-02, -3.85169327e-01, -5.67647398e-01, -5.56722224e-01,
        7.44956955e-02,  

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [8]:
print(model['nights'])

array([ 8.34019780e-02,  5.68733318e-03, -4.97448713e-01,  4.24134493e-01,
        5.19104481e-01, -2.83573598e-01, -1.61089286e-01, -3.13409492e-02,
        3.73554826e-01,  2.95112610e-01, -5.54936111e-01, -1.13063566e-02,
       -5.77557564e-01,  3.59845072e-01,  2.46771231e-01, -5.30997775e-02,
       -1.65473521e-01,  1.56806290e-01,  2.06319243e-01, -3.26517493e-01,
       -2.03059167e-01,  2.42494136e-01, -3.29739124e-01,  1.57074369e-02,
       -7.22689390e-01,  6.35452747e-01,  9.89861786e-02,  1.51157930e-01,
        3.54554296e-01,  2.68665841e-03, -5.66109478e-01,  2.02572212e-01,
        7.29037821e-02, -4.15093541e-01,  4.02934372e-01,  9.94668230e-02,
       -1.39799282e-01, -6.73641115e-02,  3.72268140e-01,  1.95475519e-01,
       -6.56093145e-03, -6.47969022e-02,  3.39752942e-01, -5.67338914e-02,
        1.08628184e-01,  1.63001820e-01, -1.22839451e-01,  1.91182479e-01,
       -1.87458098e-02, -3.33474368e-01, -4.96106029e-01, -4.83274043e-01,
        6.59496412e-02,  

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


The ``in`` operation works slightly differently from the original word2vec. It tests whether a vector for the given word exists or not, not whether the word is present in the word vocabulary. To test whether a word is present in the training word vocabulary -




Tests if word present in vocab



In [9]:
print("word" in model.wv.vocab)

False


Tests if vector present for word



In [10]:
print("word" in model)

True


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """Entry point for launching an IPython kernel.


Similarity operations
---------------------




Similarity operations work the same way as word2vec. **Out-of-vocabulary words can also be used, provided they have at least one character ngram present in the training data.**




In [11]:
print("nights" in model.wv.vocab)

False


In [12]:
print("night" in model.wv.vocab)

True


In [13]:
print(model.similarity("night", "nights"))

0.9999927


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


Syntactically similar words generally have high similarity in fastText models, since a large number of the component char-ngrams will be the same. As a result, fastText generally does better at syntactic tasks than Word2Vec. A detailed comparison is provided `here <Word2Vec_FastText_Comparison.ipynb>`_.




Other similarity operations
^^^^^^^^^^^^^^^^^^^^^^^^^^^

The example training corpus is a toy corpus, results are not expected to be good, for proof-of-concept only



In [14]:
print(model.most_similar("nights"))

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2020-03-10 21:42:33,048 : INFO : precomputing L2-norms of word weight vectors
2020-03-10 21:42:33,049 : INFO : precomputing L2-norms of ngram weight vectors


[('study', 0.9982730150222778),
 ('boat', 0.9982665777206421),
 ('often', 0.9982637166976929),
 ('Arafat', 0.9982549548149109),
 ('north.', 0.9982534646987915),
 ('"That', 0.9982510209083557),
 ('Arafat,', 0.998248815536499),
 ('Endeavour', 0.9982373118400574),
 ('stage', 0.9982361197471619),
 ('beyond', 0.9982330799102783)]


In [15]:
print(model.n_similarity(['sushi', 'shop'], ['japanese', 'restaurant']))

0.99995035


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


In [16]:
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

'breakfast'


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [17]:
print(model.most_similar(positive=['baghdad', 'england'], negative=['london']))

[('1', 0.2431073784828186),
 ('40', 0.2382356971502304),
 ('2', 0.23509719967842102),
 ('26', 0.23366421461105347),
 ('20', 0.23335930705070496),
 ('blaze', 0.23280811309814453),
 ('UN', 0.23270395398139954),
 ('keep', 0.2319527119398117),
 ('...', 0.23165921866893768),
 ('As', 0.23148386180400848)]


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [18]:
print(model.accuracy(questions=datapath('questions-words.txt')))

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.wv.accuracy() instead).
  """Entry point for launching an IPython kernel.
2020-03-10 21:42:52,953 : INFO : family: 0.0% (0/2)
2020-03-10 21:42:52,966 : INFO : gram3-comparative: 25.0% (3/12)
2020-03-10 21:42:52,974 : INFO : gram4-superlative: 8.3% (1/12)
2020-03-10 21:42:52,982 : INFO : gram5-present-participle: 20.0% (4/20)
2020-03-10 21:42:52,992 : INFO : gram6-nationality-adjective: 35.0% (7/20)
2020-03-10 21:42:53,001 : INFO : gram7-past-tense: 5.0% (1/20)
2020-03-10 21:42:53,009 : INFO : gram8-plural: 8.3% (1/12)
2020-03-10 21:42:53,012 : INFO : total: 17.3% (17/98)


[{'correct': [], 'incorrect': [], 'section': 'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': 'capital-world'},
 {'correct': [], 'incorrect': [], 'section': 'currency'},
 {'correct': [], 'incorrect': [], 'section': 'city-in-state'},
 {'correct': [],
  'incorrect': [('HE', 'SHE', 'HIS', 'HER'), ('HIS', 'HER', 'HE', 'SHE')],
  'section': 'family'},
 {'correct': [], 'incorrect': [], 'section': 'gram1-adjective-to-adverb'},
 {'correct': [], 'incorrect': [], 'section': 'gram2-opposite'},
 {'correct': [('GOOD', 'BETTER', 'GREAT', 'GREATER'),
              ('GREAT', 'GREATER', 'LOW', 'LOWER'),
              ('LONG', 'LONGER', 'GREAT', 'GREATER')],
  'incorrect': [('GOOD', 'BETTER', 'LONG', 'LONGER'),
                ('GOOD', 'BETTER', 'LOW', 'LOWER'),
                ('GREAT', 'GREATER', 'LONG', 'LONGER'),
                ('GREAT', 'GREATER', 'GOOD', 'BETTER'),
                ('LONG', 'LONGER', 'LOW', 'LOWER'),
                ('LONG', 'LONGER', 'GOOD', 'BETTER'),
  

Word Movers distance
^^^^^^^^^^^^^^^^^^^^

Let's start with two sentences:



In [19]:
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

Remove their stopwords.




In [21]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
sentence_obama = [w for w in sentence_obama if w not in stopwords]
sentence_president = [w for w in sentence_president if w not in stopwords]

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\user/nltk_data'
    - 'c:\\users\\user\\appdata\\local\\programs\\python\\python36\\nltk_data'
    - 'c:\\users\\user\\appdata\\local\\programs\\python\\python36\\share\\nltk_data'
    - 'c:\\users\\user\\appdata\\local\\programs\\python\\python36\\lib\\nltk_data'
    - 'C:\\Users\\user\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


Compute WMD.



In [22]:
distance = model.wmdistance(sentence_obama, sentence_president)
print(distance)

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.


ModuleNotFoundError: No module named 'pyemd'

That's all! You've made it to the end of this tutorial.




In [23]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('fasttext-logo-color-web.png')
imgplot = plt.imshow(img)
plt.axis('off')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'fasttext-logo-color-web.png'